<a href="https://colab.research.google.com/github/ShaswataJash/kfpcomponent/blob/main/TabularDataPreparationUsingPycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is the development workflow for kubeflow pipeline component of the same name as this notebook. Refer https://github.com/ShaswataJash/kfpcomponent

#Install required softwares

In [1]:
!uname -a

Linux 108a0453d1b0 5.4.188+ #1 SMP Sun Apr 24 10:03:06 PDT 2022 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [3]:
!python --version

Python 3.7.13


In [1]:
!pip install pycaret==2.3.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 5.0 MB/s 
     |████████████████████████████████| 6.8 MB 31.2 MB/s 
     |████████████████████████████████| 17.8 MB 40.0 MB/s 
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 10.4 MB 27.6 MB/s 
     |████████████████████████████████| 1.3 MB 40.6 MB/s 
     |████████████████████████████████| 1.7 MB 36.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 167 kB 67.4 MB/s 
     |████████████████████████████████| 2.0 MB 40.2 MB/s 
     |████████████████████████████████| 120 kB 68.5 MB/s 
     |████████████████████████████████| 262 kB 39.5 MB/s 
     |████████████████████████████████|

In [2]:
#Ref: https://github.com/pycaret/pycaret/issues/2490
!pip install Jinja2==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 5.2 MB/s 
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!apt-get install ca-certificates fuse tzdata curl unzip && \
  echo "user_allow_other" >> /etc/fuse.conf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fuse is already the newest version (2.9.7-1ubuntu1).
ca-certificates is already the newest version (20211016~18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.18).
tzdata is already the newest version (2022a-0ubuntu0.18.04).
tzdata set to manually installed.
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!curl https://rclone.org/install.sh | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4497  100  4497    0     0   8236      0 --:--:-- --:--:-- --:--:--  8221
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone  [binary]
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/git-log.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.58.1-linux-amd64/rclone.1  [text]  
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Purging old database entries in /usr/share/man/it...
Processing manual pages under /usr/share/man/it...
Purging old database entries

In [5]:
!rclone --version

rclone v1.58.1
- os/version: ubuntu 18.04 (64 bit)
- os/kernel: 5.4.188+ (x86_64)
- os/type: linux
- os/arch: amd64
- go/version: go1.17.9
- go/linking: static
- go/tags: none


#Develop source code files

In [6]:
%%writefile data_preparation.py
#!/usr/bin/env python3
import argparse

# Defining and parsing the command-line arguments
parser = argparse.ArgumentParser(description='kubeflow pipeline component to read csv file and prepare the data')
parser.add_argument('--bypass-rclone-for-input-data', default=False, action="store_true", help='whether input csv file should be read like local file - rclone is completely bypassed')
parser.add_argument('--bypass-rclone-for-output-data', default=False, action="store_true", help='whether output csv file should be written like local file - rclone is completely bypassed')
parser.add_argument('--rclone-environment-var', type=str, default= '{}', help='json formatted key-value pairs of strings which will be set as environment variables before executing rclone commands')
parser.add_argument('--input-datasource-directory-mountable', default=False, action="store_true", help='whether input csv file is present in mountable remote location')
parser.add_argument('--input-datasource-file-name', type=str, help='name of the csv file including file extension and the directory/bucket path holding the specific file(if any)')
parser.add_argument('--additional-options-csv-parsing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.read_csv()')
parser.add_argument('--type-of-data-analysis-task', choices=['classification', 'regression', 'clustering', 'anomaly_detection'])
parser.add_argument('--target-variable-name', type=str, help='for classification and regression, specify the column name holding target variable')
parser.add_argument('--target-emptyindicator', type=str, default='', help='if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc')
parser.add_argument('--data-preparations-options', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pycaret setup() function')
parser.add_argument('--additional-options-csv-writing', type=str, default= '{}', help='json formatted key-value pairs of strings which will be passed to pandas.to_csv()')
parser.add_argument('--output-datasource-directory-mountable', default=False, action="store_true", help='whether output csv file will be written in mountable remote location')
parser.add_argument('--output-datasource-file-name', type=str, help='filename of the prepared data including the directory/bucket path holding the specific file(if any)')
args = parser.parse_args()

#sanity check of arguments
if args.bypass_rclone_for_input_data:
    args.input_datasource_directory_mountable = False

if args.bypass_rclone_for_output_data:
    args.output_datasource_directory_mountable = False

if args.bypass_rclone_for_input_data and args.bypass_rclone_for_output_data:
    args.rclone_environment_var = '{}'

#setting rclone related env
import os
import json
rclone_config = json.loads(args.rclone_environment_var)
print('rclone_config = (', type(rclone_config), ')', rclone_config)
for item in rclone_config.items():
    os.environ[item[0]] = item[1]

#temporary directory creation
import tempfile
if not args.bypass_rclone_for_input_data:
    local_datastore_read_dir = tempfile.mkdtemp(prefix="my_local_read-")
    print('local_datastore_read_dir:',local_datastore_read_dir)

if not args.bypass_rclone_for_output_data:
    local_datastore_write_dir = tempfile.mkdtemp(prefix="my_local_write-")
    print('local_datastore_write_dir:',local_datastore_write_dir)

#input file handling
import subprocess
import sys
import ntpath
if args.input_datasource_directory_mountable:
    input_data_read_cmd = "rclone -v mount remoteread:" + ntpath.dirname(args.input_datasource_file_name) + ' ' + local_datastore_read_dir + ' --daemon'
else:
    input_data_read_cmd = "rclone -v copy remoteread:" + args.input_datasource_file_name + ' ' + local_datastore_read_dir
print(input_data_read_cmd)
input_data_read_call = subprocess.run(input_data_read_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
print(input_data_read_call.stdout)
if input_data_read_call.returncode != 0:
    print("Error in rclone, errorcode=", input_data_read_call.returncode)
    sys.stdout.flush()
    sys.exit("Forceful exit as rclone returned error in context of reading")

#output file handling
if args.output_datasource_directory_mountable:
    output_data_write_cmd = "rclone -v mount remotewrite:" + ntpath.dirname(args.output_datasource_file_name) + ' ' + local_datastore_write_dir + ' --daemon'
    print(output_data_write_cmd)
    output_data_write_call = subprocess.run(output_data_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(output_data_write_call.stdout)
    if output_data_write_call.returncode != 0:
        print("Error in rclone, errorcode=", output_data_write_call.returncode)
        sys.stdout.flush()
        sys.exit("Forceful exit as rclone returned error in context of mounted writing")

#handling input csv file reading
import pandas
try:
    parse_config = json.loads(args.additional_options_csv_parsing)
    parse_config['filepath_or_buffer'] =  args.input_datasource_file_name \
        if args.bypass_rclone_for_input_data else os.path.join(local_datastore_read_dir,ntpath.basename(args.input_datasource_file_name))
    print('parse_config = (', type(parse_config), ')', parse_config)
    my_data = pandas.read_csv(**parse_config)
    print(my_data)
    
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while parsing input csv file")

#handling data preprocessing
import pycaret
try:
    if os.path.exists("logs.log"):
        os.remove("logs.log") #removing any content from log which pycaret will internally use for its own logging
    print('pycaret version = ', pycaret.utils.version())
    setup_config = json.loads(args.data_preparations_options)
    if args.type_of_data_analysis_task == 'classification':
        import pycaret.classification
        setup_fn = pycaret.classification.setup
        get_config_fn = pycaret.classification.get_config
        setup_config['target'] = args.target_variable_name
        
    elif args.type_of_data_analysis_task == 'regression':
        import pycaret.regression
        setup_fn = pycaret.regression.setup
        get_config_fn = pycaret.regression.get_config
        setup_config['target'] = args.target_variable_name

    elif args.type_of_data_analysis_task == 'clustering':
        import pycaret.clustering
        setup_fn = pycaret.clustering.setup
        get_config_fn = pycaret.clustering.get_config

    elif args.type_of_data_analysis_task == 'anomaly':
        import pycaret.anomaly
        setup_fn = pycaret.anomaly.setup
        get_config_fn = pycaret.anomaly.get_config
        
    #as part of pycaret's data cleaning the rows with target column = nan are not being cleaned up. Thus, cleaning those rows explicitely
    if len(args.target_emptyindicator) > 0:
        #ref: https://stackoverflow.com/questions/49291740/delete-rows-if-there-are-null-values-in-a-specific-column-in-pandas-dataframe
        import numpy as np
        my_data[args.target_variable_name] = my_data[args.target_variable_name].replace(args.target_emptyindicator, np.nan)
        my_data = my_data.dropna(axis=0, subset=[args.target_variable_name])

    setup_config['data'] = my_data
    setup_config['log_experiment'] = False
    setup_config['data_split_shuffle'] = False
    setup_config['html'] = False
    setup_config['silent'] = True
    print('setup_config = (', type(setup_config), ')', setup_config)
    setup_fn(**setup_config)
    #ref: https://www.kdnuggets.com/2020/11/5-things-doing-wrong-pycaret.html
    X_transformed = get_config_fn('X')
    my_transformed_data = X_transformed
    if args.type_of_data_analysis_task == 'classification' or args.type_of_data_analysis_task == 'regression':
        y_transformed = get_config_fn('y')
        my_transformed_data = X_transformed.merge(y_transformed,left_index=True, right_index=True)
    
    print("====== PREPARED DATA ====")
    print(my_transformed_data)
    print("=========================")

    pycaret.utils.get_system_logs() #this will print the pycaret's own log into console
    
except BaseException as err:
    pycaret.utils.get_system_logs()
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while transforming input dataframe")

#handling output csv file writing
try:
    to_csv_config = json.loads(args.additional_options_csv_writing)
    to_csv_config['path_or_buf'] = args.output_datasource_file_name \
        if args.bypass_rclone_for_output_data else os.path.join(local_datastore_write_dir,ntpath.basename(args.output_datasource_file_name))
    print('to_csv_config = (', type(to_csv_config), ')', to_csv_config)
    my_transformed_data.to_csv(**to_csv_config)
except BaseException as err:
    print("Error=", err, ' ', type(err))
    sys.stdout.flush()
    sys.exit("Forceful exit as exception encountered while trying to write prepared data")

if args.bypass_rclone_for_output_data:
    sys.stdout.flush()
    sys.exit(0)

if not args.output_datasource_directory_mountable:
    output_data_write_cmd = "rclone -v copy " + os.path.join(local_datastore_write_dir,ntpath.basename(args.output_datasource_file_name)) \
        + " remotewrite:" + args.output_datasource_file_name
    print(output_data_write_cmd)
    output_data_write_call = subprocess.run(output_data_write_cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(output_data_write_call.stdout)
    if output_data_write_call.returncode != 0:
        print("Error in rclone, errorcode=", output_data_write_call.returncode)
        sys.stdout.flush()
        sys.exit("Forceful exit as rclone returned error in context of writing final csv file (copy mode)")

Writing data_preparation.py


Docker size reduction tips:


*   https://devopscube.com/reduce-docker-image-size/
*   https://www.ecloudcontrol.com/best-practices-to-reduce-docker-images-size/



In [7]:
%%writefile Dockerfile
FROM python:3.7.13-slim

RUN python3 -m pip install pycaret==2.3.10
#installing jinja2 additionally due to Ref: https://github.com/pycaret/pycaret/issues/2490
RUN python3 -m pip install Jinja2==3.1.2

#install fuse as dependency for rclone. Additionally, install curl, unzip for rclone installer to work
#libgomp1 installation for pycaret in python-slim
RUN apt-get update \
    && apt-get install --no-install-recommends -y curl fuse libgomp1 unzip \
    && echo "user_allow_other" >> /etc/fuse.conf \
    && curl https://rclone.org/install.sh | bash \
    && apt-get -y remove --purge curl unzip \
    && rm -rf /var/lib/apt/lists/* \
    && rclone --version

COPY src/data_preparation.py /tmp
COPY tests/test_validation.py /tmp
COPY run_tests.sh /tmp
RUN chmod 544 /tmp/run_tests.sh

Writing Dockerfile


In [8]:
%%writefile run_tests.sh
#!/bin/bash

mkdir /tmp/my_local_dir_for_test

#Test: csv reading source from http, rclone read in copy
python /tmp/data_preparation.py --rclone-environment-var '{"RCLONE_CONFIG_REMOTEREAD_TYPE":"http", "RCLONE_CONFIG_REMOTEREAD_URL":"https://raw.githubusercontent.com/pycaret/datasets/main/data/common/"}' \
    --input-datasource-file-name 'CTG.csv' --additional-options-csv-parsing '{"sep":"," , "header":0}' \
    --type-of-data-analysis-task 'classification' --target-variable-name 'NSP' \
    --data-preparations-options '{"ignore_low_variance":true, "remove_outliers":true, "remove_multicollinearity":true, "multicollinearity_threshold":0.7}' \
    --bypass-rclone-for-output-data --output-datasource-file-name '/tmp/my_local_dir_for_test/CTG_data-prep.csv' \
    --additional-options-csv-writing '{"index":false}'

#https://registry.opendata.aws/humor-detection/
#Test: csv reading source from s3(AWS provider), rclone read in mount
python /tmp/data_preparation.py --rclone-environment-var '{"RCLONE_CONFIG_REMOTEREAD_TYPE":"s3", "RCLONE_CONFIG_REMOTEREAD_PROVIDER":"AWS", "RCLONE_CONFIG_REMOTEREAD_REGION":"us-west-2"}' \
    --input-datasource-directory-mountable --input-datasource-file-name 'humor-detection-pds/Non-humours-biased.csv' \
    --type-of-data-analysis-task 'classification' --target-variable-name 'label' \
    --data-preparations-options '{"preprocess":false, "ignore_features":["image_url"]}' \
    --bypass-rclone-for-output-data --output-datasource-file-name '/tmp/my_local_dir_for_test/Non-humours-biased_data-prep.csv' \
    --additional-options-csv-writing '{"index":false}'

python /tmp/test_validation.py

Writing run_tests.sh


In [9]:
%%writefile test_validation.py
#!/usr/bin/env python3
import pandas
df = pandas.read_csv(filepath_or_buffer = '/tmp/my_local_dir_for_test/CTG_data-prep.csv')
assert len(df.index) == 2126 #original data had 2129 rows, amongst that 3 rows have no target
assert df.isnull().sum().sum() == 0 #pycaret will remove all missing values

df = pandas.read_csv(filepath_or_buffer = '/tmp/my_local_dir_for_test/Non-humours-biased_data-prep.csv')
assert len(df.columns) == 3 #original data had 4 columns, amongst that one will be dropped
print ('test-validation done successfully')

Writing test_validation.py


Refer: https://github.com/RealOrangeOne/docker-rclone-mount/blob/master/docker-compose.yml

If we have to use mount feature of rclone, it needs to have fuse support in underneath linux kernel. For that we are adding SYS_ADMIN in capability. But note without using mount feature also, we can do testing. in that case, rclone will use only copy feature.

In [10]:
%%writefile docker-compose.test.yml
services:
  sut:
    build: .
    command: /tmp/run_tests.sh
    cap_add:
      - SYS_ADMIN
    security_opt:
      - apparmor:unconfined
    devices:
      - "/dev/fuse:/dev/fuse"

Writing docker-compose.test.yml



*   https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#designing-a-pipeline-component
*   https://github.com/kubeflow/pipelines/blob/sdk/release-1.8/sdk/python/kfp/dsl/types.py



In [17]:
%%writefile component.yaml
name: TabularDataPreparationUsingPycaret
description: |
    Prepare tabular data (csv file) using pycaret library. (For pycaret's data pre-processing capabilities, refer https://pycaret.gitbook.io/docs/get-started/preprocessing)
    Refer data-preparations-options in command line arguments. 
    pycaret internally uses pandas dataframe to read and write csv file. You can utilize options exposed by panda's read_csv() and to_csv(). 
    Refer additional-options-csv-parsing and additional-options-csv-writing in command line arguments
    Input and output csv files can be stored in rclone compatible storage. Both mount and copy mode are supported. (refer: https://rclone.org/)
    rclone configurations have to be shared through environment variables (refer: https://rclone.org/docs/#environment-variables). 
    Create rclone read and write configuration file name as 'REMOTEREAD' and 'REMOTEWRITE'. Because the same are used within code.
    So convention for creating any environment variables related to rclone should start either with 'RCLONE_CONFIG_REMOTEREAD' or 'RCLONE_CONFIG_REMOTEWRITE'.
    There are options to bypass rclone separately for input and output datasource. Such bypassing will be useful for the scenario like vertexAI pipelines where
    an external system emulates cloud storage (like gcs, minio, s3) as locally mounted storage.

inputs:
- {name: bypass-rclone-for-input-data, type: Boolean, default: false, description: 'whether input csv file should be read like local file - rclone is completely bypassed'}
- {name: bypass-rclone-for-output-data, type: Boolean, default: false, description: 'whether output csv file should be written like local file - rclone is completely bypassed'}
- {name: rclone-environment-var, type: String, default: '{}', description: 'json formatted key-value pairs of strings which will be set as environment variables before executing rclone commands'}
- {name: input-datasource-directory-mountable, type: Boolean, default: false, description: 'whether input csv file is present in mountable remote location'}
- {name: input-datasource-file-name, type: String, description: 'name of the csv file including file extension and the directory/bucket path holding the specific file(if any)'}
- {name: additional-options-csv-parsing, type: String, default: '{}', description: 'json formatted key-value pairs of strings which will be passed to pandas.read_csv()'}
- {name: type-of-data-analysis-task, type: String, description: 'choice amongst classification, regression, clustering, anomaly_detection'}
- {name: target-variable-name, type: String, description: 'for classification and regression, specify the column name holding target variable'}
- {name: target-emptyindicator, type: String, default: '', description: 'if target variable column holds null or na, those rows will be dropped. Sometime empty can be indicated by other representative string like - or *** etc'}
- {name: data-preparations-options, type: String, default: '{}', description: 'json formatted key-value pairs of strings which will be passed to pycaret setup() function'}
- {name: additional-options-csv-writing, type: String, default: '{}', description: 'json formatted key-value pairs of strings which will be passed to pandas.to_csv()'}
- {name: output-datasource-directory-mountable, type: Boolean, default: false, description: 'whether output csv file will be written in mountable remote location'}

outputs:
- {name: output-datasource-file-name, type: String, description: 'filename of the prepared data including the directory/bucket path holding the specific file(if any)'}

implementation:
  container:
    image: hub.docker.com/shasjash/kfpcomponents/TabularDataPreparationUsingPycaret_devlatest
    # command is a list of strings (command-line arguments). 
    # The YAML language has two syntaxes for lists and you can use either of them. 
    # Here we use the "flow syntax" - comma-separated strings inside square brackets.
    command: [
      python3, 
      # Path of the program inside the container
      /tmp/data_preparation.py,
      --bypass-rclone-for-input-data,
      {inputValue: bypass-rclone-for-input-data},
      --bypass-rclone-for-output-data,
      {inputValue: bypass-rclone-for-output-data},
      --rclone-environment-var,
      {inputValue: rclone-environment-var},
      --input-datasource-directory-mountable,
      {inputValue: input-datasource-directory-mountable},
      --input-datasource-file-name, 
      {inputPath: input-datasource-file-name},
      --additional-options-csv-parsing, 
      {inputValue: additional-options-csv-parsing},
      --type-of-data-analysis-task, 
      {inputValue: type-of-data-analysis-task},
      --target-variable-name, 
      {inputValue: target-variable-name},
      --target-emptyindicator, 
      {inputValue: target-emptyindicator},
      --data-preparations-options, 
      {inputValue: data-preparations-options},
      --additional-options-csv-writing, 
      {inputValue: additional-options-csv-writing},
      --output-datasource-directory-mountable, 
      {inputValue: output-datasource-directory-mountable},
      --output-datasource-file-name, 
      {outputPath: output-datasource-file-name},
    ]

Overwriting component.yaml


#Software testing

Now let us simulate testing what will be done by docker-hub infrastructure as part of auto-testing by using docker-compose.test.yml present in github source repository.

In [12]:
!rm -rf /tmp/my_local_dir_for_test
!chmod 544 run_tests.sh
!cp data_preparation.py /tmp
!cp test_validation.py /tmp
!./run_tests.sh

rclone_config = ( <class 'dict'> ) {'RCLONE_CONFIG_REMOTEREAD_TYPE': 'http', 'RCLONE_CONFIG_REMOTEREAD_URL': 'https://raw.githubusercontent.com/pycaret/datasets/main/data/common/'}
local_datastore_read_dir: /tmp/my_local_read-889t2gmd
rclone -v copy remoteread:CTG.csv /tmp/my_local_read-889t2gmd
2022/06/19 17:45:20 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
2022/06/19 17:45:20 INFO  : CTG.csv: Copied (new)
2022/06/19 17:45:20 INFO  : 
Transferred:   	  277.715 KiB / 277.715 KiB, 100%, 0 B/s, ETA -
Transferred:            1 / 1, 100%
Elapsed time:         0.3s


parse_config = ( <class 'dict'> ) {'sep': ',', 'header': 0, 'filepath_or_buffer': '/tmp/my_local_read-889t2gmd/CTG.csv'}
          FileName  ...  NSP
0     Variab10.txt  ...  2.0
1       Fmcs_1.txt  ...  1.0
2       Fmcs_1.txt  ...  1.0
3       Fmcs_1.txt  ...  1.0
4       Fmcs_1.txt  ...  1.0
...            ...  ...  ...
2124  S8001045.dsp  ...  2.0
2125  S8001045.dsp  ...  1.0
2126       

In [13]:
! pip3 install kfp==1.8.12

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 301 kB 5.0 MB/s 
     |████████████████████████████████| 106 kB 54.0 MB/s 
     |████████████████████████████████| 1.6 MB 24.3 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 57 kB 6.1 MB/s 
     |████████████████████████████████| 56 kB 5.6 MB/s 
     |████████████████████████████████| 87 kB 7.3 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
  Created wheel for kfp: filename=kfp-1.8.12-py3-none-any.whl size=419048 sha256=d5da7f673e850bcd957d33959ece0a37d4ea2c4afb1a8f8e5f7cff2f67a0d726
  Stored in directory: /root/.cache/pip/wheels/54/0c/4a/3fc55077bc88cc17eacaae34c5fd3f6178c1d16d2ee3b0afdf
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=d08b600becefa47ec3308748b772210e33786c81429551eb239dca109d7a1450
  Stored in directory: /root/.cache/pip/whee

First validate the component.yaml file in http://www.yamllint.com/. Once component.yaml file is corrected, execute the below cell to finally check

In [18]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

kfp.components.load_component_from_file('component.yaml')


<function TabularDataPreparationUsingPycaret>

#Push the code to github

Before commiting code to github, install github client (gh) by following instruction mentioned in https://github.com/cli/cli/blob/trunk/docs/install_linux.md (Choose Debian, Ubuntu Linux way of installation) 

Use the colab's 'Terminal' icon present in left vertical pane to open linux terminal to type commands. Once 'gh' is installed, type **$gh auth login** (refer https://docs.github.com/en/get-started/getting-started-with-git/caching-your-github-credentials-in-git) to follow onscreen prompts. For colab, use **Paste an authentication token** option. Personal tokens can be generated in https://github.com/settings/tokens

You can use Shift+Ctrl+v shortcut to paste any string in colab console

In [19]:
!pwd

/content


In [20]:
!rm -Rf kfpcomponent

In [21]:
!git clone https://github.com/ShaswataJash/kfpcomponent.git

Cloning into 'kfpcomponent'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 174 (delta 90), reused 112 (delta 46), pack-reused 0
Receiving objects: 100% (174/174), 74.67 KiB | 3.73 MiB/s, done.
Resolving deltas: 100% (90/90), done.


Follow directory structure according to https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#organizing-the-component-files

In [22]:
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret/src
!mkdir kfpcomponent/TabularDataPreparationUsingPycaret/tests

mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret’: File exists
mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret/src’: File exists
mkdir: cannot create directory ‘kfpcomponent/TabularDataPreparationUsingPycaret/tests’: File exists


In [23]:
#it will ensure file is coped in git repo only if file content is changed by checking checksum of file content
!rsync -c data_preparation.py kfpcomponent/TabularDataPreparationUsingPycaret/src
!rsync -c component.yaml kfpcomponent/TabularDataPreparationUsingPycaret/component.yaml
!rsync -c test_validation.py kfpcomponent/TabularDataPreparationUsingPycaret/tests
!rsync -c Dockerfile kfpcomponent/TabularDataPreparationUsingPycaret/
!rsync -c run_tests.sh kfpcomponent/TabularDataPreparationUsingPycaret/
!rsync -c docker-compose.test.yml kfpcomponent/TabularDataPreparationUsingPycaret/

In [24]:
%cd kfpcomponent

/content/kfpcomponent


In [25]:
!git add -A

In [26]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   TabularDataPreparationUsingPycaret/component.yaml
	modified:   TabularDataPreparationUsingPycaret/run_tests.sh
	modified:   TabularDataPreparationUsingPycaret/src/data_preparation.py



For git-user who has set their email visibility as private, git provides alternate email address to use in web-based Git operations, e.g., edits and merges. The alias email can be viewed in https://github.com/settings/emails

In [27]:
!git config --global user.email "29448766+ShaswataJash@users.noreply.github.com"

In [28]:
!git commit -a -m "redesigned to support both kubeflow and vertexAI pipelines"

[main 2f81ae4] redesigned to support both kubeflow and vertexAI pipelines
 3 files changed, 88 insertions(+), 60 deletions(-)
 rewrite TabularDataPreparationUsingPycaret/run_tests.sh (63%)


In [29]:
!git push origin main

Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (7/7), 2.17 KiB | 2.17 MiB/s, done.
Total 7 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/ShaswataJash/kfpcomponent.git
   4cf01c1..2f81ae4  main -> main


In [30]:
%cd ..

/content
